In [1]:
from sagemaker.amazon.record_pb2 import Record
import io
import numpy as np
import struct


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/xdg-ubuntu/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/pappa/.config/sagemaker/config.yaml


In [3]:
sources = (np.random.random((5000, 10)) * 100).astype(np.int32)
targets = (np.random.random((5000, 5)) * 100).astype(np.int32)

sources.shape, targets.shape, sources.dtype, targets.dtype

((5000, 10), (5000, 5), dtype('int32'), dtype('int32'))

In [8]:
def write_recordio(f, data):
    kmagic = 0xced7230a
    length = len(data)
    f.write(struct.pack('I', kmagic))
    f.write(struct.pack('I', length))
    upper_align = ((length + 3) >> 2) << 2
    padding = bytes([0x00 for _ in range(upper_align - length)])
    f.write(data)
    f.write(padding)


def list_to_record_bytes(source: list[int] = None, target: list[int] = None):
    record = Record()
    record.features['source'].int32_tensor.values.extend(source)
    record.features['target'].int32_tensor.values.extend(target)
    return record.SerializeToString()

def to_proto(f, sources, targets):
    for source, target in zip(sources, targets):
        record = list_to_record_bytes(source, target)
        write_recordio(f, record)

In [9]:

f = io.BytesIO()
to_proto(f, sources, targets)
f.seek(0)

0